In [1]:
# Install required libraries
!pip install pytesseract opencv-python transformers torch torchvision -q
!apt-get install -y tesseract-ocr

import pytesseract
import cv2
import torch
import re
import os
import numpy as np
from transformers import LayoutLMv3Processor, LayoutLMv3ForTokenClassification
from google.colab import files
from PIL import Image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following N

In [2]:
# Load LayoutLMv3 Processor and Model
processor = LayoutLMv3Processor.from_pretrained("microsoft/layoutlmv3-base")
model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-base")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/856 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Preprocess the image for better OCR accuracy
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
    image = cv2.resize(image, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)  # Resize for better clarity
    image = cv2.GaussianBlur(image, (5, 5), 0)  # Reduce noise
    _, image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)  # Binarization
    return image

In [4]:
# Extract text from the processed image
def extract_text_from_image(image_path):
    processed_image = preprocess_image(image_path)
    text = pytesseract.image_to_string(processed_image, config='--psm 6')  # Use page segmentation mode 6
    return text

In [5]:
# Extract relevant details using fine-tuned regex patterns
def extract_entities(text):
    extracted_info = {"Employee Name": "", "Designation": "", "Total Earnings": "", "Net Pay": ""}

    # Refined regex patterns for accurate data extraction
    name_match = re.search(r"Employee\s*name\s*:\s*([A-Za-z\s]+)", text, re.IGNORECASE)
    designation_match = re.search(r"Designation\s*:\s*([A-Za-z\s]+)", text, re.IGNORECASE)
    total_earnings_match = re.search(r"Total Earnings\s*[\n\s]*([\d,]+)", text, re.IGNORECASE)
    net_pay_match = re.search(r"Net Pay\s*[\n\s]*([\d,]+)", text, re.IGNORECASE)

    if name_match:
        extracted_info["Employee Name"] = name_match.group(1).strip()
    if designation_match:
        extracted_info["Designation"] = designation_match.group(1).strip()
    if total_earnings_match:
        extracted_info["Total Earnings"] = total_earnings_match.group(1).replace(',', '').strip()
    if net_pay_match:
        extracted_info["Net Pay"] = net_pay_match.group(1).replace(',', '').strip()

    return extracted_info

In [6]:

# Upload and process the payslip image
def process_payslip():
    uploaded = files.upload()
    file_path = list(uploaded.keys())[0]

    # Extract text from the processed image
    text = extract_text_from_image(file_path)

    # Extract relevant details
    extracted_data = extract_entities(text)

    return extracted_data

In [9]:
# Run the extraction process
print(" Please upload a payslip (JPG image):")
extracted_data = process_payslip()
print("\n Extracted Payslip Details:")
print(extracted_data)

 Please upload a payslip (JPG image):


Saving payslip.webp to payslip (2).webp

 Extracted Payslip Details:
{'Employee Name': 'Sally Harley\nPay Period', 'Designation': 'Marketing Executive\nWorked Days', 'Total Earnings': '', 'Net Pay': '9500'}
